**Cyclistic** - Google Data Analytics Certification Capstone Project  
Surya Bhardwaj    


How Does a Bike-Share Navigate Speedy Success?  

**Background:**
This project is for the Google Data Analytics Certification capstone project.  


This project will be completed by using the 6 Data Analytics stages:  

Ask: Identify the business task and determine the key stakeholders.  
Prepare: Collect the data, identify how it’s organized, determine the credibility of the data.  
Process: Select the tool for data cleaning, check for errors and document the cleaning process.  
Analyze: Organize and format the data, aggregate the data so that it’s useful, perform calculations and identify trends and relationships.  
Share: Use design thinking principles and data-driven storytelling approach, present the findings with effective visualization. Ensure the analysis has answered the business task.  
Act: Share the final conclusion and the recommendations.  


**Ask:**  

Business Task:  
Recommend marketing strategies aimed at converting casual riders into annual members by better understanding how annual members and casual riders use Cyclistic bikes differently.     

Stakeholders:  
Lily Moreno: The director of marketing and my manager.  
Cyclistic executive team: A detail-oriented executive team who will decide whether to approve the recommended marketing program.  
Cyclistic marketing analytics team: A team of data analysts responsible for collecting, analyzing, and reporting data that helps guide Cyclistic’s marketing strategy.  

**Prepare:**  

For this project, I will use the public data of Cyclistic’s historical trip data to analyze and identify trends. The data has been made available by [Motivate International Inc.](https://divvy-tripdata.s3.amazonaws.com/index.html)  under the [license.](https://www.divvybikes.com/data-license-agreement)

I downloaded the ZIP files containing the csv files from the above link but while uploading the files in kaggle (as I am using kaggle notebook), it gave me a warning that the dataset is already available in kaggle. So I will be using the dataset **cyclictic-bike-share** dataset from kaggle. The dataset has 13 csv files from April 2020 to April 2021. For the purpose of my analysis I will use the csv files from April 2020 to March 2021. The source csv files are in Kaggle so I can rely on it's integrity.   

I am using Microsoft Excel to get a glimpse of the data. There is one csv file for each month and has information about the bike ride which contain details of the ride id, rideable type, start and end time, start and end station, latitude and longitude of the start and end stations.

**Process:**  

I will use R as language in kaggle to import the dataset to check how it’s organized, whether all the columns have appropriate data type, find outliers and if any of these data have sampling bias.

I will be using below R libraries

In [3]:
#Load the tidyverse, lubridate, ggplot2, sqldf and psych libraries

library(tidyverse)
library(lubridate)
library(ggplot2)
library(plotrix)

In [4]:
#Set the working directory 

setwd("/kaggle/input/cyclistic-bike-share")

In [ ]:
#Import the csv files

r_202004 <- read.csv("202004-divvy-tripdata.csv")
r_202005 <- read.csv("202005-divvy-tripdata.csv")
r_202006 <- read.csv("202006-divvy-tripdata.csv")
r_202007 <- read.csv("202007-divvy-tripdata.csv")
r_202008 <- read.csv("202008-divvy-tripdata.csv")
r_202009 <- read.csv("202009-divvy-tripdata.csv")
r_202010 <- read.csv("202010-divvy-tripdata.csv")
r_202011 <- read.csv("202011-divvy-tripdata.csv")
r_202012 <- read.csv("202012-divvy-tripdata.csv")
r_202101 <- read.csv("202101-divvy-tripdata.csv")
r_202102 <- read.csv("202102-divvy-tripdata.csv")
r_202103 <- read.csv("202103-divvy-tripdata.csv")

In [ ]:
#Preview and check the structure of the data

str(r_202004)
str(r_202005)
str(r_202006)
str(r_202007)
str(r_202008)
str(r_202009)
str(r_202010)
str(r_202011)
str(r_202012)
str(r_202101)
str(r_202102)
str(r_202103)


From the structure I see that all 12 csvs have common attributes. Once basic data type is cleaning is done at individual dataframe, I will then merge and do further cleaning.  


The 'started_at' and 'ended_at' attributes which represents the start and end ride time is currently set as character data type in all dataframes. I will change these 2 attributes to date data type.  

Also, the 'start_station_id' and 'end_station_id' in r_202012, r_202101, r_202102 and r_202103 dataframes is currently set as character data type. I will change these to integer data type so that these 2 attribues' data type is consistent across the 12 dataframes.  


In [ ]:
#Change the 'start_station_id' attribute in r_202012, r_202101, r_202102 and r_202103 dataframes to integer data type

#Supress warnings

options(warn=-1)

r_202012$start_station_id <- as.integer(r_202012$start_station_id)
r_202101$start_station_id <- as.integer(r_202101$start_station_id)
r_202102$start_station_id <- as.integer(r_202102$start_station_id)
r_202103$start_station_id <- as.integer(r_202103$start_station_id)  



#Change the 'end_station_id' attribute in r_202012, r_202101, r_202102 and r_202103 dataframes to integer data type  

r_202012$end_station_id <- as.integer(r_202012$end_station_id)
r_202101$end_station_id <- as.integer(r_202101$end_station_id)
r_202102$end_station_id <- as.integer(r_202102$end_station_id)
r_202103$end_station_id <- as.integer(r_202103$end_station_id)




In [ ]:
 #Merge individual month's data frames into one big data frame

all_trips <- bind_rows(r_202004, r_202005, r_202006, r_202007, r_202008, r_202009, r_202010, r_202011, r_202012, r_202101, r_202102, r_202103)

In [ ]:
#Check the structure and get a glimpse of the consolidated data frame

str(all_trips)

glimpse(all_trips)

nrow(all_trips)

In [ ]:
# Add columns that list the date, month, day, and year of each ride
# This will allow to aggregate ride data for each month, day, or year 


all_trips$date <- as.Date(all_trips$started_at)
all_trips$month <- format(as.Date(all_trips$date), "%B")
all_trips$day <- format(as.Date(all_trips$date), "%d")
all_trips$year <- format(as.Date(all_trips$date), "%Y")
all_trips$day_of_week <- weekdays(all_trips$date)


In [ ]:
#Check member_casual and rideable_type column to if there are any discrepancies on the rider and bike type

unique(all_trips$member_casual)

unique(all_trips$rideable_type)


The observations have proper values in these 2 columns

In [ ]:
#Remove observations with NA

all_trips <- drop_na(all_trips)


In [ ]:
#Check for test stations

unique(all_trips$start_station_name[grep("TEST", all_trips$start_station_name)])


In [ ]:
#Check for test stations
unique(all_trips$start_station_name[grep("test", all_trips$start_station_name)])

In [ ]:
#Remove the test stations and any stations with blank

all_trips <- (filter(all_trips, !(start_station_name == "WATSON TESTING - DIVVY" | start_station_name == "HUBBARD ST BIKE CHECKING (LBS-WH-TEST)" | start_station_name =="hubbard_test_lws" | start_station_name =="")))

In [ ]:
#Create a data frame of stations with start station name and latitude and longitude. I will use this data frame while analyzing the popular top 10 stations

all_trips_stations <- all_trips[,c(5,9,10)]

In [ ]:
#Remove duplicates based on the station name

all_trips_stations <- all_trips_stations[!duplicated(all_trips_stations$start_station_name),]

In [ ]:
#Find total stations

NROW(unique(all_trips_stations))

In [ ]:
#Calculate the ride length and convert it to Minutes

all_trips$ride_length <- difftime(all_trips$ended_at, all_trips$started_at)

all_trips$ride_length <- all_trips$ride_length/60

all_trips$ride_length <- round(all_trips$ride_length, 2)


In [ ]:
# Inspect the structure of the columns

str(all_trips)


In [ ]:
# Converting "ride_length" to numeric so that I can run calculations on the data

all_trips$ride_length <- as.numeric(as.character(all_trips$ride_length))



In [ ]:
#Remove observations where the ride length is less than 0

all_trips <- filter(all_trips, ride_length > 0)

In [ ]:
# Find the overall ride_lenth min, max, mean and median by rider type

summary(all_trips$ride_length)

# Compare members and casual users
all_trips %>% 
  group_by(member_casual) %>% 
  summarise(avg_ride_lenth = mean(ride_length), median_ride_length = median(ride_length), max_ride_length = max(ride_length), min_ride_length = min(ride_length))

In [ ]:
# Order the days of the week. Will also order the month

all_trips$day_of_week <- ordered(all_trips$day_of_week, levels=c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"))

all_trips$month <- ordered(all_trips$month, levels=c("January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"))

In [ ]:
# Check the average ride time by each day for members vs casual users

aggregate(all_trips$ride_length ~ all_trips$member_casual + all_trips$day_of_week, FUN = mean)

In [ ]:
#All rides by day of the week and type of rider

all_trips_by_day <- all_trips %>% 
  group_by(member_casual, day_of_week) %>% 
  summarise(number_of_rides = n(), average_duration = mean(ride_length)) %>%
  arrange(member_casual, day_of_week)

In [ ]:
#View the data frame

head(all_trips_by_day,14)

In [ ]:
# Turn off scientific notation

options(scipen=999)

setwd("/kaggle/working")

#Analyze ridership data by type and weekday

all_trips %>% 
  group_by(member_casual, day_of_week) %>% 
  summarise(number_of_rides = n(), average_duration = mean(ride_length)) %>%
  arrange(member_casual, day_of_week)%>% 
  ggplot(aes(x = day_of_week, y = number_of_rides, fill = member_casual)) +
  geom_col(position = "dodge") + labs(title="Total Number of Rides by Day", x = "Week Day", y = "Number of Rides") + theme(axis.text.x = element_text(angle = 60, hjust = 1))

In [ ]:
#Find the total casual rides on Saturday, Sunday compared to Monday through Friday

total_rides_casual_sat_sun <- NROW(filter(all_trips, member_casual == "casual" & (day_of_week == "Saturday" | day_of_week == "Sunday")))

total_rides_casual_sat_sun

total_rides_casual_mon_fri <- NROW(filter(all_trips, member_casual == "casual" & !(day_of_week == "Saturday" | day_of_week == "Sunday")))

total_rides_casual_mon_fri

In [ ]:
#Percentage of casual riders on Saturday, Sunday compared to Monday through Friday

labs <- c("Mon-Friday", "Sat-Sun")
slices_casual_week <- c(total_rides_casual_mon_fri, total_rides_casual_sat_sun)
piepercent <- round(100 * slices_casual_week / sum(slices_casual_week), 1)
lbls <- paste(labs, piepercent)
lbls_casual_week <- paste(lbls, "%", sep="")

lbls_casual_week

In [ ]:
# Analyze ridership data by type and month

all_trips %>% 
  group_by(member_casual, month) %>% 
  summarise(number_of_rides = n(), average_duration = mean(ride_length)) %>% 
  arrange(member_casual, month)	%>% 
  ggplot(aes(x = month, y = number_of_rides, fill = member_casual)) +
  geom_col(position = "dodge") + labs(title="Total Number of Ride by Month", x = "Month", y = "Number of Rides") + theme(axis.text.x = element_text(angle = 60, hjust = 1))

In [ ]:
#Find the total casual rides by month - July to September compared to rest of the year

total_rides_casual_jul_sept <- NROW(filter(all_trips, member_casual == "casual" & (month == "July" | month == "August" | month == "September")))

total_rides_casual_jul_sept

total_rides_casual_except_Jul_sept <- NROW(filter(all_trips, member_casual == "casual" & !(month == "July" | month == "August" | month == "September")))

total_rides_casual_except_Jul_sept

In [ ]:
#Percentage of casual riders by month - July to September compared to rest of the year

labs <- c("Jul To Sept", "Rest of the Year")
slices_casual_month <- c(total_rides_casual_jul_sept, total_rides_casual_except_Jul_sept)
piepercent <- round(100 * slices_casual_month / sum(slices_casual_month), 1)
lbls <- paste(labs, piepercent)
lbls_casual_month <- paste(lbls, "%", sep="")


lbls_casual_month

In [ ]:
#Find the total member rides by month - July to September compared to rest of the year

total_rides_member_jul_sept <- NROW(filter(all_trips, member_casual == "member" & (month == "July" | month == "August" | month == "September")))

total_rides_member_jul_sept

total_rides_member_except_Jul_sept <- NROW(filter(all_trips, member_casual == "member" & !(month == "July" | month == "August" | month == "September")))

total_rides_member_except_Jul_sept

In [ ]:
#Percentage of member riders by month - July to September compared to rest of the year

labs <- c("Jul To Sept", "Rest of the Year")
slices_member_month <- c(total_rides_member_jul_sept, total_rides_member_except_Jul_sept)
piepercent <- round(100 * slices_member_month / sum(slices_member_month), 1)
lbls <- paste(labs, piepercent)
lbls_member_month <- paste(lbls, "%", sep="")


lbls_member_month

In [ ]:
#Create a new data frame for member riders

all_trips_member <-  filter(all_trips, member_casual == "member")

In [ ]:
#Analyze member ridership data by type and month

all_trips_member %>% 
  group_by(rideable_type, month) %>% 
  summarise(number_of_rides = n(), average_duration = mean(ride_length)) %>% 		
  arrange(rideable_type, month)	%>% 
  ggplot(aes(x = month, y = number_of_rides, fill = rideable_type)) +
  geom_col(position = "dodge") + labs(title="Total Number of Member Rides by Month", x = "Month", y = "Number of Member Rides") + theme(axis.text.x = element_text(angle = 60, hjust = 1))

In [ ]:
# Analyze member ridership data by type and month


all_trips_member %>% 
  group_by(rideable_type, day_of_week) %>% 
  summarise(number_of_rides = n(), average_duration = mean(ride_length)) %>% 
  arrange(rideable_type, day_of_week)%>% 
  ggplot(aes(x = day_of_week, y = number_of_rides, fill = rideable_type)) +
  geom_col(position = "dodge") + labs(title="Total Number of Member Rides by Day", x = "Week Day", y = "Number of Member Rides") + theme(axis.text.x = element_text(angle = 60, hjust = 1))

In [ ]:
#Create a new data frame for casual riders

all_trips_casual <-  filter(all_trips, member_casual == "casual")

In [ ]:
#Analyze casual ridership data by type and month

all_trips_casual %>% 
  group_by(rideable_type, month) %>% 
  summarise(number_of_rides = n(), average_duration = mean(ride_length)) %>% 		
  arrange(rideable_type, month)	%>% 
  ggplot(aes(x = month, y = number_of_rides, fill = rideable_type)) +
  geom_col(position = "dodge") + labs(title="Total Number of Casual Rides by Month", x = "Month", y = "Number of Casual Rides") + theme(axis.text.x = element_text(angle = 60, hjust = 1))

In [ ]:
# Analyze casual ridership data by type and month


all_trips_casual %>% 
  group_by(rideable_type, day_of_week) %>% 
  summarise(number_of_rides = n(), average_duration = mean(ride_length)) %>% 
  arrange(rideable_type, day_of_week)%>% 
  ggplot(aes(x = day_of_week, y = number_of_rides, fill = rideable_type)) +
  geom_col(position = "dodge") + labs(title="Total Number of Casual Rides by Day", x = "Week Day", y = "Number of Casual Rides") + theme(axis.text.x = element_text(angle = 60, hjust = 1))

Find the most popular route and stations of casual rides

In [ ]:
#Create a new column with route on the casual riders data frame

all_trips_casual <- all_trips_casual %>%
  mutate(route = paste(start_station_name, "To", sep=" "))
         
         
all_trips_casual <- all_trips_casual %>%       
  mutate(route = paste(route, end_station_name, sep =" "))

In [ ]:
#Find the most popular route by number of casual rides

popular_ride_route <- all_trips_casual %>% 
  group_by(route) %>%
  summarise(number_of_rides  = n(), average_duration_minutes = mean(ride_length)) %>% 
  arrange(route, number_of_rides, average_duration_minutes)

In [ ]:
#Create a data frame of top 10 routes of casual riders 

popular_ride_route_top10 <- head(arrange(popular_ride_route, desc(number_of_rides)),10)

head(popular_ride_route_top10, 10)

I have observed that the start and end station are same for the top 10 casal riders' route. I will cross-check this for one of the station

In [ ]:
#Vefiry one of the number of observations with same start and end station for one of the station

NROW(filter(all_trips_casual, start_station_name == "Millennium Park" & end_station_name == "Millennium Park"))

**The start and end stations are same for the Top 10 route of casual riders.**    

In [ ]:
# Separate the top 10 start and end station names

popular_ride_route_top10 <- popular_ride_route_top10 %>%
  separate(route, c("start_station_name", "end_station_name"), sep = " To ")

In [ ]:
# Create a new data frame with top 10 stations, number of rides and average ride duration

popular_ride_route_top10_start <- popular_ride_route_top10[,c(1,3,4)]

In [ ]:
#Create a new data frame by joining all_trips_stations and popular_ride_route_top10_start to find the latitude and longitude of top 10 stations

m_top10_stations_casual <- merge(popular_ride_route_top10_start, all_trips_stations)

In [ ]:
#View the top 10 stations' latitude and longitude

head(m_top10_stations_casual, 10)

I will export the **m_top10_stations_casual** data frame and then **import in** **Tableau** for further analysis as it has the details about the latitude and longitude of the stations.

![Cyclistic Popular Stations_Casual_Riders](https://storage.googleapis.com/d101-analytics/Top%20%2010%20Stations%20With%20Maximum%20Casual%20Riders%20-%20April%2C2020%20To%20March%2C%202021.png)

Find the top 10 routes and stations of member riders

In [ ]:
#Create new data frame containing only member riders

all_trips_member <- filter(all_trips, member_casual == "member")



#Create a new column with route

all_trips_member <- all_trips_member %>%
  mutate(route = paste(start_station_name, "To", sep=" "))
         
         
all_trips_member <- all_trips_member %>%       
  mutate(route = paste(route, end_station_name, sep =" "))

In [ ]:
#Most popular route by number of rides

popular_ride_route_member <- all_trips_member %>% 
  group_by(route) %>%
  summarise(number_of_rides  = n(), average_duration_minutes = mean(ride_length)) %>% 
  arrange(route, number_of_rides, average_duration_minutes)



In [ ]:
#Top 10 popular member ride routes

popular_ride_route_member_top10 <- head(arrange(popular_ride_route_member, desc(number_of_rides)),10)

head(popular_ride_route_member_top10, 10)

In [ ]:
#Vefiry one of the number of observations with same start and end station for one of the station

NROW(filter(all_trips_member, start_station_name == "Lake Shore Dr & Belmont Ave" & end_station_name == "Lake Shore Dr & Belmont Ave"))

**The start and end stations are same for the Top 10 route of member riders.**

In [ ]:
# Separate the top 10 start and end station names

popular_ride_route_member_top10 <- popular_ride_route_member_top10 %>%
  separate(route, c("start_station_name", "end_station_name"), sep = " To ")

In [ ]:
# Create a new data frame with top 10 stations, number of rides and average ride duration

popular_ride_route_top10_member_start <- popular_ride_route_member_top10[,c(1,3,4)]
#Create a new data frame by joining all_trips_stations and popular_ride_route_top20_start to find the latitude and longitude of top 20 stations

m_top10_stations_member <- merge(popular_ride_route_top10_member_start, all_trips_stations)

head(m_top10_stations_member, 10)


I will export the **m_top_stations_member** and then **import in Tableau** for further analysis as it has the details about the latitude and longitude of the stations.

![Cyclistic Popular Stations_Member_Riders](https://storage.googleapis.com/d101-analytics/Cyclistic%20Popular%20Stations%20-%20Member%20Riders.png)

**Share:**     


The average ride duration of ~46 minutes for **casual rider** is around **3 times higher compared** to that of member rider of ~16 minutes.  
 
  
The average ride duration for **member rider is consisteny almost throughout the week** with Monday to Friday at ~16 minutes with a slight increase to ~18 minutes over Saturday and Sunday.  
 
The average ride duration for **casual rider is almost consistent throughout the week** with Monday to Saturday at ~45 minutes; with a slight increase to ~50 minutes on Sunday.  
 
The total number of ride is almost constant throughout 7 days for member riders whereas **~42% of the total ride for casual riders are on Saturday and Sunday only.**   


In [ ]:

pie3D(slices_casual_week, labels = lbls_casual_week, explode = 0.1, col= terrain.colors(2), main = "Casual Riders Breakup by Day: Sat-Sun Vs Mon-Fri")

**July to September constitute to more than 50% of the overall total rides** for both casual and member riders; with ~61% for casual riders and ~52% for member riders.

In [ ]:
 pie3D(slices_casual_month, labels = lbls_casual_month, explode = 0.1, col= terrain.colors(2), main = "Casual Riders Breakup by Month: July-September Vs Rest of the Year")

**The top 10 routes' start station with maximum number of rides for both casual and member riders are differenet.**

![Top 10 Stations With Maximum Riders](https://storage.googleapis.com/d101-analytics/Cyclistic%20Top%2010%20Stations%20With%20Maximum%20Riders.png)

The top 10 routes' start and end stations are same for casual riders. These **top 10 stations also corresponds to the popular sites in Chigago** as per Microsoft Bing search *Popular sites in Chicago.* [Reference](https://www.bing.com/maps?q=popular+sites+in+chicago&go=Search&qs=ds&form=QBRE)  

[Top 10 Casual Rider Stations Tableau Dashboard Link](https://public.tableau.com/app/profile/moirangthem.arup.singh/viz/Cyclistic-HowDoesaBike-ShareNavigateSpeedySuccess/CyclisticPopularStations-CasualRiders)

For both member and casual rider's ride '*docked bike*' is the most preferred bike almost throughout the year except from Decemember to March where '*classic bike*' was most popular.

**Act:**  
The high level recommendations for increasing the number of rider membership are:  

A little more than *60% of the casual riders* use Cyclictic over a span of 3 months from *July to September* which are the busy months with visitors in Chicago with a lot of events and festivals; **Marketing team must focus on top 10 stations during these months which are around the popular tourist's site** for:     
       ****A weekly, monthly and quarterly membership schemes***       
      ****Partner with Google and Microsoft search engines and popular visitor's sites of interest to increase the reach and offer special packages on the membership schemes***     
      ****Increasing the number of bikes' availability over the period of these 3 months***  
        
    

Design campaign to ***promote the usage of bikes regularly by highliting the health and environmental beneficts by creating a mobile application*** which can show the calories burnt and carbon emmission reduction. And have a reward system once a certain level is reached. This will encourage memberships.  

**Collaborate with organizations and companies** in promoting the usage of bikes by offering various membership schemes; this will be a win-win situation for both.   

Increase the number of *classic bikes* availability from December to March as the total number of casual rides with classic bikes were high during these months. Design campaigns to promote the classic bike's usage by highliting it's benefits during these coldest months of Chicago.    

Further analysis needs to be carried out to find insights to reduce the average ride duration of casual riders as currently although ~43% of the total rides are done by casual riders but the average ride duration is ~3 times more than the member riders. And to increase the availability of bikes; need to shorten the ride duration.     

  
  
    
    
      
      
  
    
    
	


Finally, ***Thank You*** to the Google Data Analytics coursera team for meticulously planning and delivering such an amazing course and giving an opportunity to work on the project. Thanks to the R community for all the great documentation across forums which helped me to choose the right tools and funtions which helped in prepare, process and analyze phases.  
  
  